<div style="position:relative; height:160px;">
<img src="logoDECATUFS.png" width="500" height="500" style="position:absolute; left:-8px;">
</div>

<div style="text-align:center;font-size:17px; white-space:pre;margin-top:-35px">
        <h3>             Modelos Lineares Generalizados</h3>
        Disciplina ofertada pelo DECAT/UFS
        Código: ESTAT0092
        Nível: Graduação
        Carga horária: 60h
        Período: 2020.2
        Professor responsável e ministrante: Luiz Henrique Dore
        <h3>            Aula 13: avaliação da significância individual dos coeficientes de regressão</h3>
</div>

# Exemplo 3

<p style="text-align: justify; font-size:16px">
Considere o conjunto de dados <b>icu.csv</b>, descrito no exemplo 3 da aula 2. Os objetivos são os seguintes: ajustar um modelo de regressão logística, no qual a variável resposta é a variável <b>sta</b> (Sobrevivência) e a variável preditora é a variável <b>age</b> (Idade); calcular o intervalo de confiança, com 99% de confiança, para cada um dos coeficientes de regressão.
<p>

### 3.1 Leitura e visualização dos dados

In [1]:
dados<-read.csv("/Dados/Google_Drive_UFS/Aulas/2020_2/MLG/Dados/icu.csv",header=TRUE)
n<-dim(dados)[1] #Número de observações
head(dados) #Visualizando as primeiras observações

,id,sta,age,gender,race,ser,can,crn,inf,cpr,⋯,hra,pre,type,fra,po2,ph,pco,bic,cre,loc
,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,4,Died,87,Female,White,Surgical,No,No,Yes,No,⋯,96,No,Emergency,Yes,<= 60,< 7.25,> 45,>= 18,<= 2.0,Nothing
2,8,Lived,27,Female,White,Medical,No,No,Yes,No,⋯,88,No,Emergency,No,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing
3,12,Lived,59,Male,White,Medical,No,No,No,No,⋯,80,Yes,Emergency,No,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing
4,14,Lived,77,Male,White,Surgical,No,No,No,No,⋯,70,No,Elective,No,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing
5,27,Died,76,Female,White,Surgical,No,No,Yes,No,⋯,90,Yes,Emergency,No,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing
6,28,Lived,54,Male,White,Medical,No,No,Yes,No,⋯,103,No,Emergency,Yes,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing


In [2]:
sta<-as.numeric(dados[,"sta"]=="Lived") #0 para Died e 1 para Lived
age<-dados[,3]

### 3.2 Formulação do modelo

<p style="text-align: justify; font-size:15px">
    <ol style="font-size:16px">
        <li>$Sobrevivência\sim Bernoulli(\mu)$;</li>
        <li>$\eta=\beta_0+\beta_1\cdot Idade$;</li>
        <li>$\ln\left(\frac{\mu}{1-\mu}\right)=\eta$.</li>
    </ol>
</p>

### 3.3 Estimação dos parâmetros por máxima verossimilhança usando a função ``glm``.

<p style="text-align: justify;font-size: 16px">Estimação dos coeficientes de regressão.</p>

In [3]:
aj<-glm(formula=sta~age,family=binomial(link="logit"),data=dados)
beta<-aj$coefficients

<p style="text-align: justify;font-size: 16px">Visualizando as estimativas dos parâmetros.</p>

In [4]:
print(beta)

(Intercept)         age 
 3.05851323 -0.02754261 


### 1.4 Executando o teste de Wald e calculando os intervalos de confiança manualmente

<p style="text-align: justify; font-size:16px">Gerando o vetor dos valores observados da resposta e a matriz design.</p>

In [5]:
y<-sta
X<-age
X<-cbind(1,X)
colnames(X)<-c("","Idade")
head(X)

,Idade
1,87
1,27
1,59
1,77
1,76
1,54


<p style="text-align: justify; font-size:16px">A estatística de Wald é $T_{W_j}=\frac{\hat\beta_j}{\sqrt{\widehat{Var(\beta_j)}}}$, onde $\widehat{Var(\hat\beta_j)}$ é o elemento $j+1$ da diagonal de $\boldsymbol{K}^{\ \ \ -1}_{\boldsymbol{\widehat\beta}\boldsymbol{\widehat\beta}}$. No caso da distribuição de Bernoulli, com ligação canônica, tem-se (ver exemplo 3 da aula 5)
    $$\boldsymbol{K}_{\boldsymbol{\beta}\boldsymbol{\beta}}=X^TVX,\ \ \ \hbox{onde}\ \ \ V=\hbox{diag}\{\mu_1(1-\mu_1),\cdots,\mu_n(1-\mu_n)\}\ \ \ \hbox{e}\ \ \ \mu_i=\frac{e^{\eta_i}}{1+e^{\eta_i}},\quad\forall\ i\in\{1,\cdots,n\}.$$
Logo, $\boldsymbol{K}^{\ \ \ -1}_{\boldsymbol{\widehat\beta}\boldsymbol{\widehat\beta}}=(X^T\widehat{V}X)^{-1}$, com $\boldsymbol{\widehat\beta}=\hbox{diag}\{\hat\mu_1,\cdots\hat\mu_n\}$ e $\hat\mu_i=\frac{e^{\hat\eta_i}}{1+e^{\hat\eta_i}}$, $\forall\ i\in\{1,\cdots,n\}$.
</p>

<p style="text-align: justify;font-size: 16px">Calculando $Var(\beta_j)$, para cada $j\in\{0,1,\cdots,p\}$. O vetor <font face="'Courier New'">betaVar</font> contém as variâncias calculadas.</p>

In [6]:
eta<-as.numeric(X%*%beta) #Estimativa de máxima verossimilhança do preditor linear
mu<-exp(eta)/(1+exp(eta)) #Estimativa de máxima verossimilhança das médias
V<-diag(mu*(1-mu)) #Calculando a estimativa da matriz V das funções de variância
Kbb<-t(X)%*%V%*%X #Calcuando Kbb
KbbInv<-solve(Kbb) #Calcuando a inversa de Kbb
betaVar<-diag(KbbInv) #Recolhendo a diagonal da inversa de Kbb, cujos valores correspondem às variâncias

<p style="text-align: justify;font-size: 16px">Visualizando os valores de $Var(\beta_j)$, para cada $j\in\{0,1,\cdots,p\}$.</p>

In [7]:
print(betaVar)

                    Idade 
0.4845858141 0.0001116123 


<p style="text-align: justify;font-size: 16px">Calculando e visualizando os valores de $T_{W_j}$, para cada $j\in\{0,1,\cdots,p\}$.</p>

In [8]:
tw<-beta/sqrt(betaVar)
print(tw)

(Intercept)         age 
   4.393646   -2.607049 


<p style="text-align: justify;font-size: 16px">Calculando e visualizando os p-valores do teste de Wald para avaliação da significância individual de $\beta_j$, $j\in\{0,1,\cdots,p\}$.</p>

In [9]:
pvalor<-2*(1-pnorm(abs(tw)))
print(pvalor)

 (Intercept)          age 
1.114655e-05 9.132640e-03 


<p style="text-align: justify;font-size: 16px">O p-valor correspondente ao coeficiente da variável <b>age</b> é menor do que 1%. Portanto, ao nível de 1% significância, o teste de Wald indica esse coeficiente é significativamente diferente de zero.</p>

<p style="text-align: justify;font-size: 16px">Calculando e visualizando os intervalos de confiança, de acordo com a fórmula
$$IC(\beta_j;1-\alpha)=\left(\hat\beta_j-z_{\left(1-\frac{\alpha}{2}\right)}\sqrt{\widehat{Var(\hat\beta_j)}};\ \hat\beta_j+z_{\left(1-\frac{\alpha}{2}\right)}\sqrt{\widehat{Var(\hat\beta_j)}}\right).$$
</p>

In [10]:
alpha=0.01 #Valor de alpha correspondente ao intervalo com nível de confiança 1-alpha
z<-qnorm(1-alpha/2) #Quantil de ordem 1-alpha/2 da distribuição N(0,1)
IC<-cbind(beta-z*sqrt(betaVar),beta+z*sqrt(betaVar)) #Calculando os intervlos de confiança.
print(IC)

                  [,1]          [,2]
(Intercept)  1.2654218  4.8516046345
age         -0.0547554 -0.0003298215


<p style="text-align: justify;font-size: 16px">O intervalo de confiança, com 99% de confiança, do coeficientes da variável <b>age</b> não contém o valor 0. Pode-se concluir que, ao nível de 1% significância, o teste de Wald indica que esse coeficiente é significativamente diferente de zero.</p>

### 1.5 Executando o teste de Wald utilizando a função ``summary``.

<p style="text-align: justify;font-size: 16px">A função <font face="'Courier New'">summary</font> realiza o teste de Wald para avaliação da significância individual dos coeficientes de regressão. Os dois principais argumentos da função são os seguintes:</p>
    <ol style="text-align: justify; font-size:16px">
        <li><font face="'Courier New'">object</font>: modelo ajustado. Saída da função <font face="'Courier New'">glm</font>.</li>
        <li><font face="'Courier New'">phi</font>: estimativa de máxima veroossimilhança de $\phi$.</li>
    </ol>
</p>

<p style="text-align: justify;font-size: 16px">Não é preciso informar o parâmetro de dispersão, pois, como se sabe, no caso da distribuição de Bernoulli, o parâmetro de dispersão é constante e igual a 1.</p>

In [11]:
summary(object=aj)


Call:
glm(formula = sta ~ age, family = binomial(link = "logit"), data = dados)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2854   0.3905   0.6145   0.7391   0.9536  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  3.05851    0.69608   4.394 1.11e-05 ***
age         -0.02754    0.01056  -2.607  0.00913 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 200.16  on 199  degrees of freedom
Residual deviance: 192.31  on 198  degrees of freedom
AIC: 196.31

Number of Fisher Scoring iterations: 4


<p style="text-align: justify;font-size: 16px">Para cada $j\in\{0,1,\cdots,p\}$, tem-se que
        <ol style="text-align: justify; font-size:16px">
        <li>a coluna <font face="'Courier New'">Estimate</font> contém o valor de $\hat\beta_j$;</li>
        <li>a coluna <font face="'Courier New'">Std. Error</font> contém o valor de $\widehat{Var(\hat\beta_j)}$;</li>
            <li>a coluna <font face="'Courier New'">z value</font> contém o valor de $t_{W_j}$;</li>
            <li>a coluna <font face="'Courier New'">Pr(>|z|)</font> contém o p-valor do teste de Wald para $\beta_j$.</li>
    </ol>
</p>

<p style="text-align: justify;font-size: 16px">Tanto o p-valor correspondente ao coeficiente da variável <b>distConc</b>, quanto o p-valor correspondente ao coeficiente da variável <b>distLoja</b> são menores do que 5%. Portanto, ao nível de 5% significância, o teste de Wald indica que ambos os coeficientes são significativamente diferentes de zero.</p>

### 2.6 Calculando os intervalos de confiança utilizando a função ``confint.default``.

<p style="text-align: justify;font-size: 16px">A função <font face="'Courier New'">confint.default</font> calcula os intervalos de confiança para os coeficientes de regressão, aproximando a distribuição da quantidade $\frac{\hat\beta_j-\beta}{\widehat{Var(\hat\beta_j)}}$ pela distribuição $N(0,1)$. Os dois principais argumentos dessa função são:</p>
    <ol style="text-align: justify; font-size:16px">
        <li><font face="'Courier New'">object</font>: modelo ajustado. Saída da função <font face="'Courier New'">glm</font>.</li>
        <li><font face="'Courier New'">level</font>: confiança do intervalo. Valor de $1-\alpha$.</li>
    </ol>
</p>

In [12]:
confint.default(object=aj,level=0.99)

,0.5 %,99.5 %
(Intercept),1.26552679,4.8514996798
age,-0.05475409,-0.0003311277


<p style="text-align: justify;font-size: 16px">O intervalo de confiança, com 99% de confiança, do coeficientes da variável <b>age</b> não contém o valor 0. Pode-se concluir que, ao nível de 1% significância, o teste de Wald indica que esse coeficiente é significativamente diferente de zero.</p>